In [1]:
import pandas as pd
passengers = pd.read_csv("train.csv")
print(passengers.shape)
print(passengers.head())

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

In [2]:
passengers['Sex'] = passengers['Sex'].map({'female':1,'male':0})
passengers['Age'].fillna(value=passengers['Age'].mean(), inplace=True)

In [3]:
passengers['FirstClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 1 else 0)
passengers['SecondClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 2 else 0)

In [4]:
features = passengers[['Sex', 'Age', 'FirstClass', 'SecondClass']]
survival = passengers['Survived']

In [5]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, survival)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train_features, train_labels)

LogisticRegression()

In [8]:
print(model.score(train_features, train_labels))

0.8068862275449101


In [9]:
from art.attacks.evasion import DeepFool
from art.estimators.classification.scikitlearn import SklearnClassifier

classifier = SklearnClassifier(model=model, clip_values=(0, 1))
attack = DeepFool(classifier)
x_test_adv = attack.generate(train_features)

DeepFool:   0%|          | 0/668 [00:00<?, ?it/s]

In [11]:
print(model.score(x_test_adv, train_labels))
print(model.score(train_features, train_labels))

0.19311377245508982
0.8068862275449101


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=66)


In [42]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1, max_iter=10000)
model.fit(X_train, y_train)

model.score(X_train, y_train), model.score(X_test, y_test)

(0.9577464788732394, 0.965034965034965)

In [43]:
from art.attacks.evasion import DeepFool
from art.estimators.classification.scikitlearn import SklearnClassifier

classifier = SklearnClassifier(model=model, clip_values=(0, 1))
attack = DeepFool(classifier)
x_test_adv = attack.generate(X_train)

DeepFool:   0%|          | 0/426 [00:00<?, ?it/s]

In [44]:
print(model.score(x_test_adv, y_train))
print(model.score(X_train, y_train))

0.6220657276995305
0.9577464788732394


In [1]:
import tensorflow as tf
import numpy as np

num_classes = 10
num_features = 784

learning_rate =0.01
training_steps = 1000
batch_size = 256
display_step =50

In [2]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train, x_test = np.array(x_train, np.float32), np.array(x_test,np.float32)
x_train, x_test = x_train.reshape([-1,num_features]), x_test.reshape([-1,num_features])
x_train ,x_test = x_train / 255., x_test / 255.

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [5]:
W = tf.Variable(tf.ones([num_features, num_classes]),name="weight")
b = tf.Variable(tf.zeros([num_classes]),name="bias")

In [6]:
def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x,W)+b)

In [7]:
def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true,depth=num_classes)
    y_pred = tf.clip_by_value(y_pred,1e-9,1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true*tf.math.log(y_pred)))

In [8]:
def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred,1),tf.cast(y_true,tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [9]:
optimizer = tf.optimizers.SGD(learning_rate)

In [10]:
def run_optimization(x,y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred,y)
        
    gradients = g.gradient(loss,[W,b])
    optimizer.apply_gradients(zip(gradients,[W,b]))

In [11]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps),1):
    run_optimization(batch_x, batch_y)
    
    if step % display_step ==0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" %(step,loss,acc))

step: 50, loss: 95.550308, accuracy: 0.886719
step: 100, loss: 90.961868, accuracy: 0.917969
step: 150, loss: 77.395538, accuracy: 0.921875
step: 200, loss: 95.116203, accuracy: 0.906250
step: 250, loss: 307.977661, accuracy: 0.824219
step: 300, loss: 106.023407, accuracy: 0.898438
step: 350, loss: 56.456673, accuracy: 0.917969
step: 400, loss: 86.889999, accuracy: 0.902344
step: 450, loss: 69.920273, accuracy: 0.925781
step: 500, loss: 80.974678, accuracy: 0.914062
step: 550, loss: 50.220078, accuracy: 0.953125
step: 600, loss: 89.388519, accuracy: 0.917969
step: 650, loss: 88.457214, accuracy: 0.882812
step: 700, loss: 80.421623, accuracy: 0.925781
step: 750, loss: 56.635689, accuracy: 0.941406
step: 800, loss: 40.442513, accuracy: 0.953125
step: 850, loss: 68.488976, accuracy: 0.914062
step: 900, loss: 54.252316, accuracy: 0.937500
step: 950, loss: 65.443405, accuracy: 0.941406
step: 1000, loss: 48.631847, accuracy: 0.945312


In [12]:
pred = logistic_regression(x_test)
print("Test Accuracy: %f" %accuracy(pred,y_test))

Test Accuracy: 0.914900


In [16]:
#Importing dataset
from sklearn.datasets import fetch_openml
mnist=fetch_openml('mnist_784')
x,y=mnist['data'],mnist['target']

In [17]:
#Splitting the dataset into train and test dataset
x_train,x_test=x[:60000],x[60000:]
y_train,y_test=y[:60000],y[60000:]

In [18]:
#Training the logistic regressor model on the train dataset
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)

c:\users\좌\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [19]:
#Predicting a single value
print(lr.predict([x_test[2000]]))

['6']


In [20]:
#Matching with its corresponding true value
print([y_test[2000]])

['6']


In [23]:
#Predicting test dataset
y_pred = lr.predict(x_test)
#Finding accuracy of our model
print(accuracy_score(y_test,y_pred))

0.9255


In [36]:
from art.attacks.evasion import DeepFool
from art.estimators.classification.scikitlearn import SklearnClassifier

classifier = SklearnClassifier(model=lr, clip_values=(0, 1))
attack = DeepFool(classifier)
x_test_adv = attack.generate(x_test)

DeepFool:   0%|          | 0/10000 [00:00<?, ?it/s]

In [37]:
#Predicting test dataset
y_pred = lr.predict(x_test_adv)
#Finding accuracy of our model
print(accuracy_score(y_test,y_pred))

0.0385
